In [ ]:
# UCSB Thomas–Fermi Solver (Standalone)
# This notebook embeds the solver implementation and the Exc_data_digitized table.

import numpy as np
from dataclasses import dataclass
from typing import Optional, Union, Any
from pathlib import Path
from scipy.fft import fft2, ifft2
from scipy.optimize import basinhopping
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt


@dataclass
class SimulationConfig:
    e: float = 1.602e-19
    epsilon_0: float = 8.854e-12
    h: float = 6.62607015e-34

    B: float = 13.0
    dt: float = 30e-9
    db: float = 30e-9

    epsilon_perp: float = 3.0
    epsilon_parallel: float = 6.6

    # XC table is embedded (CSV string below)
    exc_scale: float = 1.0

    niter: int = 10
    step_size: float = 1.0
    lbfgs_maxiter: int = 1000
    lbfgs_maxfun: int = 2_000_000

    Nx: int = 64
    Ny: int = 64

    x_min_nm: float = -150.0
    x_max_nm: float = 150.0
    y_min_nm: float = -150.0
    y_max_nm: float = 150.0

    V_B: float = 0.0
    Vt_grid: Optional[np.ndarray] = None

    potential_scale: float = 1.0
    potential_offset: float = 0.0

    solver_type: str = "solverUCSB_notebook"


class ThomasFermiSolver:
    J_to_meV = 1.0 / (1.602e-22)

    def __init__(self, cfg: SimulationConfig):
        self.cfg = cfg
        self.Nx = int(cfg.Nx)
        self.Ny = int(cfg.Ny)
        self._prepare_grid()
        self._prepare_kernels()
        self._prepare_exc_table()
        self._init_density()

    def _prepare_grid(self) -> None:
        cfg = self.cfg
        self.x_min = cfg.x_min_nm * 1e-9
        self.x_max = cfg.x_max_nm * 1e-9
        self.y_min = cfg.y_min_nm * 1e-9
        self.y_max = cfg.y_max_nm * 1e-9

        self.Lx = self.x_max - self.x_min
        self.Ly = self.y_max - self.y_min
        self.dx = self.Lx / self.Nx
        self.dy = self.Ly / self.Ny
        self.dA = self.dx * self.dy
        self.A_total = self.Lx * self.Ly

        self.x = np.linspace(self.x_min, self.x_max, self.Nx)
        self.y = np.linspace(self.y_min, self.y_max, self.Ny)
        self.X, self.Y = np.meshgrid(self.x, self.y, indexing="ij")

        if cfg.Vt_grid is None:
            Vt = np.zeros((self.Nx, self.Ny), dtype=float)
        else:
            Vt = np.asarray(cfg.Vt_grid, dtype=float)
            if Vt.shape != (self.Nx, self.Ny):
                raise ValueError(f"Vt_grid must have shape {(self.Nx, self.Ny)}, got {Vt.shape}")

        kx = 2.0 * np.pi * np.fft.fftfreq(self.Nx, d=self.dx)
        ky = 2.0 * np.pi * np.fft.fftfreq(self.Ny, d=self.dy)
        KX, KY = np.meshgrid(kx, ky, indexing="ij")
        q = np.sqrt(KX**2 + KY**2)
        q[0, 0] = 1e-20

        beta = np.sqrt(cfg.epsilon_parallel / cfg.epsilon_perp)
        dt = cfg.dt
        db = cfg.db
        denom = np.sinh(beta * (dt + db) * q)
        numer = np.sinh(beta * db * q)
        Rq = numer / denom

        Vt_q = fft2(Vt)
        Phi_q = Vt_q * Rq
        Phi_from_top = np.real(ifft2(Phi_q))

        phi_offset = -cfg.V_B * dt / (dt + db)
        Phi_grid = Phi_from_top + phi_offset
        Phi_grid = Phi_grid * cfg.potential_scale + cfg.potential_offset
        self.Phi = Phi_grid

    def _prepare_kernels(self) -> None:
        cfg = self.cfg
        self.D = cfg.e * cfg.B / cfg.h
        self.ell_B = np.sqrt(cfg.h / (2.0 * np.pi * cfg.e * cfg.B))

        kx = 2.0 * np.pi * np.fft.fftfreq(self.Nx, d=self.dx)
        ky = 2.0 * np.pi * np.fft.fftfreq(self.Ny, d=self.dy)
        KX, KY = np.meshgrid(kx, ky, indexing="ij")
        q = np.sqrt(KX**2 + KY**2)
        q[0, 0] = 1e-20

        self.G_q = np.exp(-0.5 * (self.ell_B * q) ** 2)

        epsilon_hBN = np.sqrt(cfg.epsilon_perp * cfg.epsilon_parallel)
        beta = np.sqrt(cfg.epsilon_parallel / cfg.epsilon_perp)
        self.Vq = (
            cfg.e ** 2 / (4.0 * np.pi * cfg.epsilon_0 * epsilon_hBN)
            * (4.0 * np.pi * np.sinh(beta * cfg.dt * q) * np.sinh(beta * cfg.db * q))
            / (np.sinh(beta * (cfg.dt + cfg.db) * q) * q)
        )

    def _prepare_exc_table(self) -> None:
        data = np.loadtxt("data/0-data/Exc_data_new.csv", delimiter=",", skiprows=1)
        n_exc, Exc_vals = data[:, 0], data[:, 1]
        self.exc_interp = interp1d(n_exc, Exc_vals, kind="linear", bounds_error=False, fill_value="extrapolate")

    def _init_density(self) -> None:
        nu0 = np.clip(0.5 - 1.0 * (self.Phi - np.median(self.Phi)), 0.0, 1.0)
        self.nu0 = nu0.flatten()

    def gaussian_convolve(self, arr: np.ndarray) -> np.ndarray:
        arr_q = fft2(arr)
        conv_q = arr_q * self.G_q
        return np.real(ifft2(conv_q))

    def energy(self, nu_flat: np.ndarray) -> float:
        nu = nu_flat.reshape((self.Nx, self.Ny))
        nu_eff = self.gaussian_convolve(nu)
        n_eff = nu_eff * self.D
        n_q = fft2(n_eff) * self.dA
        E_C = 0.5 / self.A_total * np.sum(self.Vq * np.abs(n_q) ** 2) * self.J_to_meV
        E_phi = np.sum(-self.cfg.e * self.Phi * n_eff) * self.dA * self.J_to_meV
        E_xc = float(np.sum(self.exc_interp(nu_eff)) * self.dA * self.D) * float(self.cfg.exc_scale)
        return float(E_phi + E_xc + E_C)

    def optimise(self):
        self.energy_history = [self.energy(self.nu0.copy())]
        def _bh_callback(x, f, accept):
            if accept:
                self.energy_history.append(float(f))
        bounds = [(0.0, 1.0)] * (self.Nx * self.Ny)
        result = basinhopping(
            self.energy,
            self.nu0.copy(),
            minimizer_kwargs={
                "method": "L-BFGS-B",
                "bounds": bounds,
                "options": {"maxiter": self.cfg.lbfgs_maxiter, "maxfun": self.cfg.lbfgs_maxfun, "ftol": 1e-6, "eps": 1e-8},
            },
            niter=self.cfg.niter,
            stepsize=self.cfg.step_size,
            disp=True,
            callback=_bh_callback,
        )
        self.nu_opt = result.x.reshape((self.Nx, self.Ny))
        self.nu_smoothed = self.gaussian_convolve(self.nu_opt)
        self.optimisation_result = result
        return result

    def plot_results(self, save_dir: Union[Path, str, None] = None, show: bool = False, title_extra: str = ""):
        if not hasattr(self, "nu_smoothed"):
            raise RuntimeError("Run optimise() before plotting results.")
        extent_lB = (self.x_min / self.ell_B, self.x_max / self.ell_B, self.y_min / self.ell_B, self.y_max / self.ell_B)
        if save_dir is not None:
            Path(save_dir).mkdir(parents=True, exist_ok=True)
        fig1 = plt.figure(figsize=(6,5))
        plt.imshow(self.nu_smoothed.T, extent=extent_lB, origin="lower", cmap="inferno", aspect="auto", vmin=0.0, vmax=1.0)
        plt.title("Optimised Filling Factor ν(r)\n" + title_extra)
        plt.xlabel("x [l_B]")
        plt.ylabel("y [l_B]")
        plt.colorbar(label="ν")
        plt.tight_layout()
        if save_dir is not None:
            fig1.savefig(Path(save_dir)/"nu_smoothed.png", dpi=300)
        fig2 = plt.figure(figsize=(6,5))
        plt.imshow(self.Phi.T, extent=extent_lB, origin="lower", cmap="viridis", aspect="auto")
        plt.title("External Potential Φ(r) [V]\n" + title_extra)
        plt.xlabel("x [l_B]")
        plt.ylabel("y [l_B]")
        plt.colorbar(label="Φ [V]")
        plt.tight_layout()
        if save_dir is not None:
            fig2.savefig(Path(save_dir)/"phi.png", dpi=300)
        # Cross section (y≈0)
        y_index = int(np.argmin(np.abs(self.y)))
        x_over_lB = self.x / self.ell_B
        phi_ext_meV = (-self.cfg.e * self.Phi[:, y_index]) * self.J_to_meV
        nu_line = self.nu_smoothed[:, y_index]
        fig3, ax1 = plt.subplots(figsize=(6.2,4.5))
        ln1 = ax1.plot(x_over_lB, phi_ext_meV, color="tab:red", linewidth=2.2, label="Φ_ext [meV]")
        ax1.set_xlabel("x [l_B]")
        ax1.set_ylabel("Φ_ext [meV]", color="tab:red")
        ax2 = ax1.twinx()
        ln2 = ax2.plot(x_over_lB, nu_line, color="tab:blue", linewidth=2.0, label="ν")
        ax2.set_ylabel("ν", color="tab:blue")
        ax2.set_ylim(0.0, 1.0)
        lines = ln1 + ln2
        labels = [l.get_label() for l in lines]
        fig3.legend(lines, labels, loc="upper right")
        plt.tight_layout()
        if save_dir is not None:
            fig3.savefig(Path(save_dir)/"phi_nu_cross_section.png", dpi=300)
        if show:
            plt.show()
        else:
            plt.close(fig1); plt.close(fig2); plt.close(fig3)

# Helper to build Vt X-mask grid

def build_vt_grid(N: int, x_min_nm: float, x_max_nm: float, y_min_nm: float, y_max_nm: float, bar_width_nm: float, V_NS: float, V_EW: float) -> np.ndarray:
    x_nm = np.linspace(x_min_nm, x_max_nm, N)
    y_nm = np.linspace(y_min_nm, y_max_nm, N)
    X_nm, Y_nm = np.meshgrid(x_nm, y_nm, indexing="ij")
    sqrt2 = np.sqrt(2.0)
    d1 = np.abs(Y_nm - X_nm) / sqrt2
    d2 = np.abs(Y_nm + X_nm) / sqrt2
    half_w = bar_width_nm / 2.0
    bar_mask = (d1 <= half_w) | (d2 <= half_w)
    remaining = ~bar_mask
    ns_region = remaining & (np.abs(Y_nm) > np.abs(X_nm))
    ew_region = remaining & (np.abs(X_nm) > np.abs(Y_nm))
    Vt = np.zeros((N, N), dtype=float)
    Vt[ns_region] = V_NS
    Vt[ew_region] = V_EW
    return Vt



In [ ]:
# Example run cell

# Magnetic length domain: 40 l_B with B=13 T
B_FIELD_T = 13.0
h = 6.62607015e-34
e = 1.602e-19
l_B = np.sqrt(h / (2.0 * np.pi * e * B_FIELD_T))
half_span_nm = 20.0 * l_B * 1e9

GRID_N = 64
X_MIN_NM, X_MAX_NM = -half_span_nm, +half_span_nm
Y_MIN_NM, Y_MAX_NM = -half_span_nm, +half_span_nm

# X mask params
BAR_WIDTH_NM = 35.0
V_NS, V_EW = (0.19, 0.51)
V_B = -0.19

vt = build_vt_grid(GRID_N, X_MIN_NM, X_MAX_NM, Y_MIN_NM, Y_MAX_NM, BAR_WIDTH_NM, V_NS, V_EW)

cfg = SimulationConfig(
    Nx=GRID_N,
    Ny=GRID_N,
    B=B_FIELD_T,
    dt=30e-9,
    db=30e-9,
    x_min_nm=X_MIN_NM,
    x_max_nm=X_MAX_NM,
    y_min_nm=Y_MIN_NM,
    y_max_nm=Y_MAX_NM,
    V_B=V_B,
    Vt_grid=vt,
    niter=10,
    step_size=1.0,
    lbfgs_maxiter=1000,
    lbfgs_maxfun=2_000_000,
    exc_scale=1.8,
)

solver = ThomasFermiSolver(cfg)
res = solver.optimise()
solver.plot_results(save_dir=None, show=True, title_extra=f"V_NS={V_NS:+.2f} V, V_EW={V_EW:+.2f} V")

